In [ ]:
!git clone https://github.com/rishikksh20/Identify-Question-Type.git

Cloning into 'Identify-Question-Type'...
remote: Enumerating objects: 26, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (26/26), done.


In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 108 kB 5.9 MB/s 
     |████████████████████████████████| 2.8 MB 32.8 MB/s 
  Created wheel for keras: filename=Keras-1.0.0-py3-none-any.whl size=130498 sha256=3ea609ff939be5ccb0c0d8020efe531eb5a48839d8859cf9e4d67729535e9db0
  Stored in directory: /root/.cache/pip/wheels/b3/2c/5d/05f97a2a95bab04657f01b979b1c8cffead1da5d8a5e6e7b62
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668111 sha256=46e9ea30136edd4724c60b3071d296d922a73018a6b2d8d0c9a597e30f524a77
  Stored in directory: /root/.cache/pip/wheels/26/68/6f/745330367ce7822fe0cd863712858151f5723a0a5e322cc144
Successfully built keras theano
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.

In [ ]:
df = pd.read_csv("/content/Identify-Question-Type/label.txt",sep=",,,",header=None ,names=['question','type'])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
df.head()

,question,type
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [ ]:
df.shape

(1483, 2)

In [ ]:
df['type']=df['type'].str.strip()

In [ ]:
df['type'].unique()

array(['unknown', 'what', 'when', 'who', 'affirmation'], dtype=object)

In [ ]:
df['question'].values

array(['how did serfdom develop in and then leave russia ? ',
       'what films featured the character popeye doyle ? ',
       "how can i find a list of celebrities ' real names ? ", ...,
       'does this hose have one ? ', 'can i get it in india ? ',
       'would this work on a 2008 ford edge with a naked roof ? '],
      dtype=object)

In [ ]:
import re
df['question'] = df['question'].apply(lambda x: x.lower())
df['question'] = df['question'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [ ]:
VALIDATION_SPLIT=0.20

In [ ]:
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle as pkl
from sklearn.naive_bayes import MultinomialNB
# organize imports
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from nltk.corpus import wordnet as wn

In [ ]:
class StemTokenizer(object):
    def __init__(self):
        self.ignore_set = {'footnote', 'nietzsche', 'plato', 'mr.'}

    def __call__(self, doc):
        words = []
        for word in word_tokenize(doc):
            word = word.lower()
            w = wn.morphy(word)
            if w and len(w) > 1 and w not in self.ignore_set:
                words.append(w)
        return words

In [ ]:
stemmer = SnowballStemmer('english').stem
def stem_tokenize(text):
    return [stemmer(i) for i in word_tokenize(text)]

In [ ]:
import nltk
nltk.download('punkt')
vectorizer = CountVectorizer(analyzer='word',lowercase=True,tokenizer=stem_tokenize)
X_train = vectorizer.fit_transform(df.question.values)
with open('vectorizer.pk', 'wb') as fin:
    pkl.dump(vectorizer, fin)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
labels = df['type']

In [ ]:
# split the data into a training set and a validation set
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_train.shape[0])

x_train = X_train[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_train[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [ ]:
clf = MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB()

In [ ]:
# evaluate the model of test data
preds = clf.predict(x_val)
print(classification_report(preds,y_val))
print("Accuracy :",clf.score(x_val,y_val))

              precision    recall  f1-score   support

 affirmation       0.86      1.00      0.92        18
     unknown       0.81      0.95      0.87        40
        what       0.98      0.85      0.91       136
        when       0.52      0.92      0.67        13
         who       1.00      0.98      0.99        89

    accuracy                           0.92       296
   macro avg       0.83      0.94      0.87       296
weighted avg       0.94      0.92      0.92       296

Accuracy : 0.9155405405405406


In [ ]:
example=vectorizer.transform(["What time does the train leave"])
clf.predict(example)

array(['what'], dtype='<U11')

In [ ]:
tf_vectorizer = TfidfVectorizer(analyzer='word',lowercase=True,tokenizer=stem_tokenize)
X_train = tf_vectorizer.fit_transform(df.question.values)
with open('tf_vectorizer.pk', 'wb') as fin:
    pkl.dump(tf_vectorizer, fin)

In [ ]:
labels = df['type']
# split the data into a training set and a validation set
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_train.shape[0])

x_train = X_train[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_train[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [ ]:
clf = MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB()

In [ ]:
example=tf_vectorizer.transform(["What time does the train leave"])
clf.predict(example)

array(['what'], dtype='<U11')

In [ ]:
import keras
!pip install tensorflow
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

     |████████████████████████████████| 462 kB 5.4 MB/s 
     |████████████████████████████████| 1.4 MB 9.8 MB/s 
  Attempting uninstall: keras
    Found existing installation: Keras 1.0.0
    Uninstalling Keras-1.0.0:
      Successfully uninstalled Keras-1.0.0


In [ ]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=30

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

In [ ]:
data=df.copy()

In [ ]:
print(data['type'].value_counts())

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(data['question'].values)
X = tokenizer.texts_to_sequences(data['question'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)


what           609
who            402
unknown        272
affirmation    104
when            96
Name: type, dtype: int64


In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Y = data['type']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)


# split the data into a training set and a validation set
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X.shape[0])

x_train = X[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]


Found 3685 unique tokens.
Shape of data tensor: (1483, 30)
Shape of label tensor: (1483, 5)


In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2022-04-14 09:28:05--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2022-04-14 09:28:05--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-04-14 09:28:05--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [ ]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [ ]:
embeddings_index = {}
f = open('/content/glove.42B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 1917494 word vectors.


In [ ]:
EMBEDDING_DIM=300

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
?LSTM

In [ ]:
embed_dim = 300
lstm_out = 196

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(lstm_out, recurrent_dropout=0.25))
model.add(Dropout(0.25))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 300)           1105800   
                                                                 
 lstm (LSTM)                 (None, 196)               389648    
                                                                 
 dropout (Dropout)           (None, 196)               0         
                                                                 
 dense (Dense)               (None, 5)                 985       
                                                                 
Total params: 1,496,433
Trainable params: 390,633
Non-trainable params: 1,105,800
_________________________________________________________________
None


In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_val, y_val))


Train on 1187 samples, validate on 296 samples
Epoch 1/20
1187/1187 [==============================] - ETA: 0s - loss: 1.3569 - acc: 0.4305

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


1187/1187 [==============================] - 6s 5ms/sample - loss: 1.3569 - acc: 0.4305 - val_loss: 1.2133 - val_acc: 0.5642
Epoch 2/20
1187/1187 [==============================] - 2s 1ms/sample - loss: 0.9842 - acc: 0.6310 - val_loss: 0.8734 - val_acc: 0.6791
Epoch 3/20
1187/1187 [==============================] - 2s 1ms/sample - loss: 0.5828 - acc: 0.8029 - val_loss: 0.5271 - val_acc: 0.8378
Epoch 4/20
1187/1187 [==============================] - 1s 1ms/sample - loss: 0.3756 - acc: 0.8821 - val_loss: 0.4120 - val_acc: 0.8919
Epoch 5/20
1187/1187 [==============================] - 2s 1ms/sample - loss: 0.2550 - acc: 0.9360 - val_loss: 0.2874 - val_acc: 0.9324
Epoch 6/20
1187/1187 [==============================] - 1s 1ms/sample - loss: 0.1636 - acc: 0.9579 - val_loss: 0.2769 - val_acc: 0.9392
Epoch 7/20
1187/1187 [==============================] - 1s 1ms/sample - loss: 0.1166 - acc: 0.9739 - val_loss: 0.2506 - val_acc: 0.9358
Epoch 8/20
1187/1187 [==============================] - 1s 

In [ ]:
example = tokenizer.texts_to_sequences(["When time does the train leave"])
example = pad_sequences(example, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
print(np.argmax(model.predict(example)))
le.inverse_transform(np.array([np.argmax(model.predict(example))]))

3


array(['when'], dtype=object)